In [50]:
import pandas as pd
import numpy as np
import sys
import os
sys.path.append(os.path.abspath(".."))
from core.viz import plot_bar
from core.s3 import S3AssetManager
from core.render_svg import generate_sackoff_svg

In [51]:
notebook_name = "fazenda_efecto_adiflow_en_sackoff"
s3 = S3AssetManager(notebook_name=notebook_name)

In [52]:
def compute_sackoff(
    df: pd.DataFrame,
    cols,

) -> pd.DataFrame:

    col_adiflow = "Tiene Adiflow"
    sackoff_name = "sackoff"
    group_cols = cols + [col_adiflow]

    df_group = (
        df
        .groupby(group_cols, dropna=False)
        .agg(
            diferencia=("diff", "sum"),
            reales=("peso_real", "sum"),
            production=("Producción (Ton)", 'sum'),
            anulation=("Anulación (Ton)", 'sum'),
            sackoff_mean=('sackoff_op', 'median'),
            ops=('op', 'nunique'),

            temp1_acond_c=('temp1_acond_c', 'mean'),

            pdi=('pdi', "mean"),
            pdi_agro=('pdi_agro', "mean"),

            finos=('finos', "mean"),
            finos_agro=('finos_agro', "mean"),

            dureza=('dureza', "mean"),
            dureza_agro=('dureza_agro', "mean")
        )
        .reset_index()
    )
    df_group[sackoff_name] = np.where(
        df_group["reales"] > 0,
        df_group["diferencia"] / df_group["reales"] * 100,
        np.nan
    )

    return df_group


def build_summary_table(summary_general_cut: pd.DataFrame) -> pd.DataFrame:
    df = summary_general_cut.copy()

    # Normalizar etiquetas por si vienen como 0/1, bool, etc.
    def norm_adiflow(x):
        s = str(x).lower()
        if "con" in s and "adiflow" in s:
            return "Con Adiflow"
        if "sin" in s and "adiflow" in s:
            return "Sin Adiflow"
        if s in ["1", "si", "sí", "true"]:
            return "Con Adiflow"
        if s in ["0", "no", "false"]:
            return "Sin Adiflow"
        return s

    df["Tiene Adiflow std"] = df["Tiene Adiflow"].apply(norm_adiflow)

    fila_con = df[df["Tiene Adiflow std"] == "Con Adiflow"].iloc[0]
    fila_sin = df[df["Tiene Adiflow std"] == "Sin Adiflow"].iloc[0]

    sackoff_sin = float(fila_sin["Sackoff Prom (%)"])
    sackoff_con = float(fila_con["Sackoff Prom (%)"])
    diff_sackoff = sackoff_con - sackoff_sin  # Con - Sin

    toneladas_con = float(fila_con["Producidas (Ton)"])
    toneladas_recuperadas = toneladas_con * diff_sackoff / 100.0

    resumen = {
        "Sackoff Sin Adiflow": round(sackoff_sin, 2),
        "Sackoff Con Adiflow": round(sackoff_con, 2),
        "Diferencia Sackoff": round(diff_sackoff, 2),
        "Toneladas Producidas Con Adiflow": round(toneladas_con, 2),
        "Toneladas Recuperadas": round(toneladas_recuperadas, 2),
    }

    return pd.DataFrame([resumen])



In [53]:

def build_monthly_summary_table(
    df: pd.DataFrame,
    month_col: str = "month",
    adiflow_col: str = "Tiene Adiflow",
    produced_col: str = "Producidas (Ton)",
    sackoff_col: str = "Sackoff (%)",
    month_order=None,
) -> pd.DataFrame:
    """
    Construye tabla resumen por mes:

    Mes | Sackoff Sin Adiflow | Sackoff Con Adiflow | Diferencia Sackoff |
        Toneladas Producidas Con Adiflow | Toneladas Recuperadas
    """

    data = df.copy()

    # Normalizar etiqueta Tiene Adiflow
    def norm_adiflow(x):
        s = str(x).lower()
        if "con" in s and "adiflow" in s:
            return "Con Adiflow"
        if "sin" in s and "adiflow" in s:
            return "Sin Adiflow"
        if s in ["1", "si", "sí", "true"]:
            return "Con Adiflow"
        if s in ["0", "no", "false"]:
            return "Sin Adiflow"
        return s

    data["Tiene Adiflow std"] = data[adiflow_col].apply(norm_adiflow)

    rows = []
    for m, g in data.groupby(month_col):
        fila_con = g[g["Tiene Adiflow std"] == "Con Adiflow"]
        fila_sin = g[g["Tiene Adiflow std"] == "Sin Adiflow"]

        if fila_con.empty or fila_sin.empty:
            # si falta alguno de los dos, saltamos el mes
            continue

        # Tomamos el valor (si hubiera más de una fila, usamos el primero;
        # puedes cambiar a sum/mean si hiciera falta)
        sackoff_con = float(fila_con[sackoff_col].iloc[0])
        sackoff_sin = float(fila_sin[sackoff_col].iloc[0])
        diff_sackoff = sackoff_con - sackoff_sin  # p.p.

        toneladas_con = float(fila_con[produced_col].iloc[0])
        toneladas_recuperadas = toneladas_con * diff_sackoff / 100.0

        rows.append({
            "Mes": m,
            "Sackoff Sin Adiflow": round(sackoff_sin, 2),
            "Sackoff Con Adiflow": round(sackoff_con, 2),
            "Diferencia Sackoff": round(diff_sackoff, 2),
            "Toneladas Producidas Con Adiflow": round(toneladas_con, 2),
            "Toneladas Recuperadas": round(toneladas_recuperadas, 2),
        })

    resumen = pd.DataFrame(rows)

    # Ordenar meses si se pasa un orden
    if month_order is not None and not resumen.empty:
        cat = pd.Categorical(resumen["Mes"], categories=month_order, ordered=True)
        resumen = resumen.assign(Mes=cat).sort_values("Mes").reset_index(drop=True)

    return resumen


In [54]:
df_cap = s3.read_excel("raw/fazenda/SACK OFF FAZENDA.xlsx", sheet_name="CAP")
df_sap = s3.read_excel("raw/fazenda/SACK OFF FAZENDA.xlsx", sheet_name="SAP")
df_sap = df_sap[df_sap["cerrada o abierta"] ==1]

In [55]:
df_sap.columns = [str(x).strip() for x in df_sap.columns]
columnas = [
'Orden', 'Material', 'Descripción',
'OP CAP', 'Cantidad planificada', 'Cantidad entregada',
'Unidad de medida',"101", "102",
"122", "309", "261",
"262", "641", "642",
'Difenrencia ent des', 'DIF PRO - CONS', 'SACKOFF %',
'CONS. CAP', 'SACOKFF CAP']
#df_sap.columns = df_cap.columns.map(str)
df_cap = df_cap[df_cap["O.P."].notnull()]
df_sap = df_sap[df_sap["Orden"].notnull()]

rename = {
    'Orden': 'order',
    'Descripción': "Dieta",
    'OP CAP': "op",
    'Fecha liberacion': "date",
    'Cantidad planificada': "panificadas",
    'Cantidad entregada': "entregadas",

     "101": "code_101",
    "102": "Anulación (Ton)",
    "122": "code_122",
    "309": "code_309",
    "261": "code_261",
    "262": "code_262",
    "641": "code_641",
    "642": "code_642",

    "PRODUCCION": "Producción (Ton)",
    'DIF PRO - CONS': "diff_prod",
    'SACKOFF %': "sackoff",
    'SACK OFF PROD': "sackoff_prod",
    'cerrada o abierta': "status"

}
df_sap = df_sap.rename(columns=rename)
df_sap_dep = df_sap[rename.values()]
cls_num = [
    "panificadas",
    "entregadas",
    "code_101",
    "Anulación (Ton)",
    "code_122",
    "code_309",
    "code_261",
    "code_262",
    "code_641",
    "code_642",
    "Producción (Ton)",
    "diff_prod",

]
df_sap_dep = df_sap_dep.copy()
for c in cls_num:
    df_sap_dep[c] = pd.to_numeric(df_sap_dep[c], errors="coerce")/1000
df_sap_dep["date"] = pd.to_datetime(df_sap_dep["date"])
df_sap_dep["month"] = df_sap_dep["date"].dt.to_period("M")

df_sap_dep["op"] = pd.to_numeric(df_sap_dep["op"], errors="coerce")

df_sap_dep["diff"] = df_sap_dep["code_101"] - df_sap_dep["Producción (Ton)"] - df_sap_dep["Anulación (Ton)"]
df_sap_dep["sackoff_op"] = df_sap_dep["diff"]/df_sap_dep["Producción (Ton)"]*100
df_sap_dep["sackoff_op"] = df_sap_dep["sackoff_op"].replace([np.inf, -np.inf], 0)

In [56]:
rename_cap = {"O.P.": "op", "Peso real": "peso_real", "Peso Agua": "peso_agua"}
df_cap = df_cap.rename(columns=rename_cap)
df_cap_dep = df_cap[rename_cap.values()].copy()
for c in ["peso_real", "peso_agua"]:
    df_cap_dep[c] = pd.to_numeric(df_cap_dep[c])/1000
df_cap_dep["op"] = pd.to_numeric(df_cap_dep["op"], errors="coerce")
df_cap_dep

,op,peso_real,peso_agua
0,20825.0,84.918095,0.77720
1,20826.0,34.398669,0.31485
2,20827.0,151.944689,1.38750
3,20828.0,151.872027,1.38780
4,20829.0,30.306526,0.27750
...,...,...,...
1067,22104.0,32.116569,0.00000
1068,22105.0,72.619554,0.00000
1069,22106.0,32.081944,0.00000
1070,22110.0,34.066832,0.00000


In [57]:
df = pd.merge(
    df_sap_dep,
    df_cap_dep,
    on='op',
    how='left',
)
df["Tiene Adiflow"] = np.where(df["peso_agua"]>0, "Con Adiflow", "Sin Adiflow")
df = df[df["op"].notnull()]
df["op"] = df["op"].astype(int).astype(str)

CUT_DATE_ENSAYO = '2025-09-01'
df = df[df["date"]>=CUT_DATE_ENSAYO]

In [58]:
qa_agro = s3.read_excel("raw/fazenda/Base Fazenda.xlsx", sheet_name='CALIDAD AGROINDUSTRIA')
qa_control = s3.read_excel("raw/fazenda/Base Fazenda.xlsx", sheet_name='3_control_prod_peletizado', skiprows=3)

In [59]:

qa_control12 = qa_control.groupby(['OP CAP']).agg(
    product_name=('PRODUCTO', 'first'),
    temp1_acond_c=('TEMPERATURA DEL ACONDICIONADOR (°C) Pelet 1', 'mean'),
    #temp2_acond_c=('TEMPERATURA DEL ACONDICIONADOR (°C) Pelet 2', 'mean'),
    pdi=('DURABILIDAD (%)', 'mean'),
    dureza=('DUREZA (kg/cm2)', 'mean'),
    finos=('FINOS (%)', 'mean')
).reset_index().rename(columns={'OP CAP': 'op', 'PRODUCTO': 'product_name'})

qa_control3 = qa_control.groupby(['- OP CAP']).agg(
      product_name=('- PRODUCTO', 'first'),
    temp1_acond_c=('- TEMPERATURA DEL ACONDICIONADOR (°C) Pelet 3', 'mean'),
    pdi=('- DURABILIDAD (%)', 'mean'),
    dureza=('- DUREZA (kg/cm2)', 'mean'),
    finos=('- FINOS (%)', 'mean')
).reset_index().rename(columns={'- OP CAP': 'op',  '- PRODUCTO': 'product_name'})

qa_control_comp = pd.concat([qa_control12, qa_control3])


for cl in ['Durabilidad pellet ','Dureza pellet ', 'Finos pellet']:
    qa_agro[cl] = pd.to_numeric(qa_agro[cl], errors='coerce')

qa_agro_comp = qa_agro.groupby(['OP CAP']).agg(
    pdi_agro=('Durabilidad pellet ', 'mean'),
    dureza_agro=('Dureza pellet ', 'mean'),
    finos_agro=('Finos pellet', 'mean')).reset_index().rename(columns={'OP CAP': 'op', 'PRODUCTO': 'product_name'})


qa_control_comp["op"] = pd.to_numeric(qa_control_comp["op"], errors='coerce')
qa_control_comp = qa_control_comp[qa_control_comp["op"].notnull()]
qa_control_comp["op"] = qa_control_comp["op"].astype(int).astype(str)

qa_agro_comp["op"] =  pd.to_numeric(qa_agro_comp["op"], errors='coerce')
qa_agro_comp = qa_agro_comp[qa_agro_comp["op"].notnull()]
qa_agro_comp["op"] = qa_agro_comp["op"].astype(int).astype(str)

# TODO, bad
qa_control_comp[qa_control_comp.duplicated(subset=['op'], keep=False)]


,op,product_name,temp1_acond_c,pdi,dureza,finos
1,21577,GESTACION,72.500000,89.000000,3.000000,11.000000
7,21598,FINALIZACION,75.000000,91.000000,3.100000,9.000000
8,21599,FINALIZACION,76.000000,93.000000,3.000000,7.000000
18,21634,GESTACION,77.000000,90.000000,3.000000,10.000000
25,21669,GESTACION,75.000000,91.600000,3.250000,8.400000
28,21682,LEVANTE,80.000000,93.600000,3.200000,6.700000
29,21686,FINALIZACION,75.000000,92.000000,3.000000,7.000000
32,21704,LEVANTE,78.000000,92.400000,3.100000,7.600000
33,21705,FINALIZACION,70.325000,95.650000,3.195000,4.750000
34,21706,FINALIZACION,70.615000,94.000000,3.150000,6.000000


In [60]:
qa_control_comp = qa_control_comp.drop_duplicates(subset=['op'], keep="first")
df = pd.merge(df, qa_agro_comp, how='left', on='op')
df = pd.merge(df, qa_control_comp, how='left', on='op')

In [61]:
q_min = df["sackoff_op"].quantile(0.01)
q_max = df["sackoff_op"].quantile(0.99)

cond1 = df["sackoff_op"] >= q_min
cond2 = df["sackoff_op"] <= q_max
cond_range = cond1 & cond2

# registro que quieres mandar a df_bad
cond_op_21944 = df["op"].isin(["21944","21864"])

# filas “buenas”: dentro del rango Y NO op 21944
conds = cond_range & ~cond_op_21944

df_cut = df[conds].copy()
df_bad = df[~conds].copy()



df_group_diet = compute_sackoff(df_cut, cols=["Dieta"])
mask = (df_group_diet
        .groupby('Dieta')['Tiene Adiflow']
        .transform('nunique')
        .eq(2))

df_both_diet = (df_group_diet[mask]
           .sort_values(['Dieta', 'Tiene Adiflow']))["Dieta"].unique()

df_both_cut = df_cut[df_cut["Dieta"].isin(df_both_diet)]

In [62]:
df_bad_dep = df_bad[['date',
        'op',
        'Dieta',
        'panificadas',
       'entregadas',
        'Producción (Ton)',
        'Anulación (Ton)',
       #'peso_real',
        'diff', 'sackoff_op',
        'Tiene Adiflow',
        'pdi_agro',
       'dureza_agro',
        'finos_agro']].round(2)

df_bad_dep = df_bad_dep.rename(columns={'diff': 'Diferencia (Ton)',
                                        'panificadas': "Planificadas (Ton)",
                                        "entregadas": "Entregadas (Ton)",

                                        'sackoff_op': 'Sackoff (%)',
                           'pdi_agro': 'Pdi (%)', 'dureza_agro': 'Dureza (kg/cm2)', 'finos_agro': 'Finos (%)'})

df_bad_dep = df_bad_dep.sort_values(["date"], ascending=False)
s3.save_dataframe(df_bad_dep, "data_bad.csv")
df_bad_dep


,date,op,Dieta,Planificadas (Ton),Entregadas (Ton),Producción (Ton),Anulación (Ton),Diferencia (Ton),Sackoff (%),Tiene Adiflow,Pdi (%),Dureza (kg/cm2),Finos (%)
349,2025-11-15,21944,GESTACION ESP P,28.0,23.66,28.05,0.00,-4.39,-15.65,Con Adiflow,89.80,3.10,10.20
285,2025-11-01,21864,FLUSHING,42.0,37.92,42.09,0.00,-4.17,-9.90,Con Adiflow,94.25,3.03,5.75
231,2025-10-20,21796,GESTACION,10.0,8.07,10.03,0.00,-1.96,-19.53,Con Adiflow,91.40,3.00,8.60
208,2025-10-14,21747,LEVANTE CMC,26.0,29.26,26.04,0.00,3.22,12.39,Sin Adiflow,93.40,3.00,6.60
190,2025-10-11,21759,MACHOS ESP 113,2.0,1.33,1.99,0.00,-0.66,-33.24,Con Adiflow,90.20,3.00,9.80
179,2025-10-09,21752,REEMPLAZOS S1B,12.0,12.84,12.06,0.00,0.77,6.39,Sin Adiflow,93.30,3.60,6.70
174,2025-10-08,21739,LEVANTE CMC,26.0,26.00,26.08,16.05,4.12,15.80,Sin Adiflow,90.95,3.05,9.05
164,2025-10-06,21732,NOVILLONAS SUPLEMENTO,4.0,4.41,4.00,0.00,0.41,10.17,Con Adiflow,97.20,3.60,2.80
157,2025-10-05,21722,REEMPLAZOS S1B,26.0,26.52,24.94,0.00,1.57,6.31,Con Adiflow,90.80,3.21,9.20
79,2025-09-18,21623,PREINICIACION F1 P INMUNIDAD,56.0,45.25,56.23,0.00,-10.99,-19.54,Sin Adiflow,97.25,3.00,0.95


In [63]:
compute_sackoff(df_both_cut, cols=[])

,Tiene Adiflow,diferencia,reales,production,anulation,sackoff_mean,ops,temp1_acond_c,pdi,pdi_agro,finos,finos_agro,dureza,dureza_agro,sackoff
0,Con Adiflow,-117.203356,20813.673062,20555.382356,22.127,-0.274150,267,74.520939,93.465028,95.081256,6.512931,6.427191,3.110190,3.168507,-0.563108
1,Sin Adiflow,-91.978755,8733.301755,8743.301755,63.210,-0.600939,154,72.191538,93.955821,94.134779,6.053154,5.823065,3.102872,3.728904,-1.053196


In [64]:
df_both_cut["date"].max()

Timestamp('2025-12-16 00:00:00')

In [65]:
by_month = compute_sackoff(df_both_cut, cols=["month"])
by_month_sackoff = by_month.groupby("Tiene Adiflow").agg(
    sackoff_mean=('sackoff_mean', 'mean'),
    sackoff=('sackoff', 'mean'),
).reset_index()
con_adiflow = by_month_sackoff.loc[by_month_sackoff['Tiene Adiflow']=='Con Adiflow', "sackoff"][0]
sin_adiflow =  by_month_sackoff.loc[by_month_sackoff['Tiene Adiflow']!='Con Adiflow', "sackoff"][1]

In [66]:
summary_general_cut = compute_sackoff(df_cut, cols=[])
cols_visual = [
    'Tiene Adiflow',
    'ops',
    'production',
    'reales',
    'anulation',
    'sackoff',
    'diferencia',
    'temp1_acond_c',
    'pdi',
    'finos',
    'dureza']
for col in cols_visual:
    if col !='Tiene Adiflow':
        summary_general_cut[col] = pd.to_numeric(summary_general_cut[col], errors='coerce')
summary_general_cut = summary_general_cut[cols_visual].round(2)
summary_general_cut = summary_general_cut.rename(columns={
    'ops': 'OPs',
    'production': 'Planificadas (Ton)',
     'reales': 'Producidas (Ton)',
    'anulation': 'Anuladas (Ton)',
    'sackoff': 'Sackoff Prom (%)',
    'diferencia': "Diferencia (Ton)",
    'temp1_acond_c': 'Temp (°C)',
    'pdi': 'Pdi (%)',
    'finos': 'Finos (%)',
    'dureza': 'Dureza (Kg/cm2)',
})
#summary_general_cut.loc[summary_general_cut["Tiene Adiflow"]=="Con Adiflow", "Sackoff Prom (%)"] = con_adiflow
#summary_general_cut.loc[summary_general_cut["Tiene Adiflow"]=="Sin Adiflow", "Sackoff Prom (%)"] = sin_adiflow
s3.save_dataframe(summary_general_cut, "summary_general.csv")
summary_general_cut

,Tiene Adiflow,OPs,Planificadas (Ton),Producidas (Ton),Anuladas (Ton),Sackoff Prom (%),Diferencia (Ton),Temp (°C),Pdi (%),Finos (%),Dureza (Kg/cm2)
0,Con Adiflow,280,22029.31,22297.14,43.83,-0.60,-133.58,74.69,93.33,6.65,3.11
1,Sin Adiflow,184,9153.18,9143.18,63.21,-1.05,-96.45,72.19,93.96,6.05,3.10


In [67]:
def extract_svg_params(df: pd.DataFrame, fecha_ini: str, fecha_fin: str, pct_datos: int) -> dict:
    """
    Convierte el DataFrame de resumen en un diccionario plano de parámetros 
    para la función de generación de SVG.
    """
    # 1. Convertir a diccionario anidado usando 'Tiene Adiflow' como clave
    # Resultado: {'Con Adiflow': {'Producidas (Ton)': 2000, ...}, 'Sin Adiflow': {...}}
    data = df.set_index('Tiene Adiflow').to_dict(orient='index')
    con = data.get('Con Adiflow', {})
    sin = data.get('Sin Adiflow', {})
    ton_con_adiflow = con.get('Producidas (Ton)', 0)
    ton_sin_adiflow = sin.get('Producidas (Ton)', 0)
    sackoff_con_adiflow = con.get('Sackoff Prom (%)', 0)
    sackoff_sin_adiflow = sin.get('Sackoff Prom (%)', 0)
    mejora_pct = sackoff_con_adiflow-sackoff_sin_adiflow
    recuperadas_prom = mejora_pct*ton_con_adiflow/100
   
    
    return {
        # Toneladas
        "ton_con_adiflow": ton_con_adiflow,
        "ton_sin_adiflow": ton_sin_adiflow,

        "mejora_pct": mejora_pct, 
        "sackoff_con": sackoff_con_adiflow,
        "sackoff_sin": sackoff_sin_adiflow,
        "recuperadas_prom": recuperadas_prom, 
        
        # Temperatura
        "temp_con": con.get('Temp (°C)', 0),
        "temp_sin": sin.get('Temp (°C)', 0),
        "delta_temp": con.get('Temp (°C)', 0) - sin.get('Temp (°C)', 0),
        
        # Calidad
        "pdi_con": con.get('Pdi (%)', 0),
        "pdi_sin": sin.get('Pdi (%)', 0),
        "finos_con": con.get('Finos (%)', 0),
        "finos_sin": sin.get('Finos (%)', 0),
        
        # Metadatos externos (pasados como argumentos)
        "fecha_ini": fecha_ini,
        "fecha_fin": fecha_fin,
        "pct_datos": pct_datos,
    }

In [68]:
svg_params = extract_svg_params(
    df=summary_general_cut,
    fecha_ini=df_cut["date"].min().strftime("%d-%b"),  
    fecha_fin=df_cut["date"].max().strftime("%d-%b"),
    pct_datos=98
)
svg_params

{'ton_con_adiflow': 22297.14,
 'ton_sin_adiflow': 9143.18,
 'mejora_pct': 0.45000000000000007,
 'sackoff_con': -0.6,
 'sackoff_sin': -1.05,
 'recuperadas_prom': 100.33713000000002,
 'temp_con': 74.69,
 'temp_sin': 72.19,
 'delta_temp': 2.5,
 'pdi_con': 93.33,
 'pdi_sin': 93.96,
 'finos_con': 6.65,
 'finos_sin': 6.05,
 'fecha_ini': '01-Sep',
 'fecha_fin': '16-Dec',
 'pct_datos': 98}

In [69]:

s3_template_url = f"s3://{s3.bucket_name}/svg_template/fazenda_sackoff.svg"
output_filename = "fazenda_sackoff.svg"


svg_params = extract_svg_params(
    df=summary_general_cut,
    fecha_ini=df_cut["date"].min().strftime("%d-%b"),  
    fecha_fin=df_cut["date"].max().strftime("%d-%b"),
    pct_datos=98
)
svg_content = generate_sackoff_svg(
    template_path=s3_template_url, 
    **svg_params
)
s3.save_svg_content(svg_content, output_filename)


's3://galileo-c4e9a2f1/images/fazenda_efecto_adiflow_en_sackoff/fazenda_sackoff.svg'

In [70]:
compute_sackoff(df_cut, cols=[])

,Tiene Adiflow,diferencia,reales,production,anulation,sackoff_mean,ops,temp1_acond_c,pdi,pdi_agro,finos,finos_agro,dureza,dureza_agro,sackoff
0,Con Adiflow,-133.581844,22297.144450,22029.309844,43.829,-0.276959,280,74.687080,93.332457,94.858410,6.648562,6.574503,3.108978,3.166446,-0.599098
1,Sin Adiflow,-96.448255,9143.176255,9153.176255,63.210,-0.560940,184,72.191538,93.955821,94.134779,6.053154,5.823065,3.102872,3.728904,-1.054866


In [71]:
summary_table = build_summary_table(summary_general_cut)
s3.save_dataframe(summary_table, "summary_recuperadas.csv")
summary_table

,Sackoff Sin Adiflow,Sackoff Con Adiflow,Diferencia Sackoff,Toneladas Producidas Con Adiflow,Toneladas Recuperadas
0,-1.05,-0.6,0.45,22297.14,100.34


In [72]:
cols = ['diff', 'sackoff_op']
group_cols = ["month",'Tiene Adiflow']
# Cantidad de registros por grupo
n_por_grupo = (
    df.groupby(group_cols)
      .size()
      .rename('n_registros')
)

# Describe solo para diff y sackoff_op
stats = (
    df
    .groupby(group_cols)[cols]
    .describe(percentiles=[0.25, 0.5, 0.75])
)

# Aplanar columnas
stats.columns = [f"{col}_{stat}" for col, stat in stats.columns]

# Unir con n_registros
resumen = stats.join(n_por_grupo)
resumen

diff_count  diff_mean  diff_std   diff_min  diff_25%  \
month   Tiene Adiflow                                                         
2025-09 Con Adiflow          52.0  -0.102762  1.251226  -3.672686 -0.482699   
        Sin Adiflow          81.0  -0.596387  2.650729 -17.465216 -0.417000   
2025-10 Con Adiflow         115.0  -0.481271  1.542875  -6.049869 -0.939699   
        Sin Adiflow          36.0  -0.151081  1.298330  -4.762731 -0.538959   
2025-11 Con Adiflow          84.0  -0.667250  1.384925  -4.425972 -1.300815   
        Sin Adiflow          38.0  -0.327225  0.698402  -2.206634 -0.719696   
2025-12 Con Adiflow          36.0  -0.723404  1.839514  -7.216724 -1.115006   
        Sin Adiflow          36.0  -0.958982  1.247197  -5.338525 -1.152489   

                       diff_50%  diff_75%  diff_max  sackoff_op_count  \
month   Tiene Adiflow                                                   
2025-09 Con Adiflow    0.001250  0.659363  2.366365              52.0   
        Sin Adiflow   -0.011622  0.416220  2.976560              81.0   
2025-10 Con Adiflow   -0.163766  0.159974  5.358398             115.0   
        Sin Adiflow   -0.102718  0.074235  4.120487              36.0   
2025-11 Con Adiflow   -0.415562  0.028875  3.367139              84.0   
        Sin Adiflow   -0.137157  0.144799  0.709246              38.0   
2025-12 Con Adiflow   -0.146795  0.012775  3.819509              36.0   
        Sin Adiflow   -0.812508 -0.157468  0.773500              36.0   

                       sackoff_op_mean  sackoff_op_std  sackoff_op_min  \
month   Tiene Adiflow                                                    
2025-09 Con Adiflow          -0.209983        1.930445       -5.802248   
        Sin Adiflow          -2.257577        5.656380      -24.706820   
2025-10 Con Adiflow          -1.085768        4.518162      -33.236116   
        Sin Adiflow          -0.895480        5.483696      -16.971731   
2025-11 Con Adiflow          -0.688760        2.538344      -15.652665   
        Sin Adiflow          -0.963695        2.399436       -9.123147   
2025-12 Con Adiflow          -1.130298        3.141107      -10.761637   
        Sin Adiflow          -2.078098        3.261816      -14.342308   

                       sackoff_op_25%  sackoff_op_50%  sackoff_op_75%  \
month   Tiene Adiflow                                                   
2025-09 Con Adiflow         -0.530970        0.003046        0.700463   
        Sin Adiflow         -2.845568       -0.037205        0.736867   
2025-10 Con Adiflow         -1.579086       -0.232223        0.638541   
        Sin Adiflow         -2.557485       -0.342737        0.782212   
2025-11 Con Adiflow         -1.359271       -0.519343        0.652388   
        Sin Adiflow         -1.813109       -0.398255        0.716594   
2025-12 Con Adiflow         -1.096784       -0.274547        0.241418   
        Sin Adiflow         -2.728378       -0.982825       -0.513647   

                       sackoff_op_max  n_registros  
month   Tiene Adiflow                               
2025-09 Con Adiflow          4.610601           52  
        Sin Adiflow          5.268541           81  
2025-10 Con Adiflow         10.165998          115  
        Sin Adiflow         15.799709           36  
2025-11 Con Adiflow          6.243745           84  
        Sin Adiflow          2.562822           38  
2025-12 Con Adiflow          5.538853           36  
        Sin Adiflow          3.866243           36

In [73]:

# Estadísticos por grupo (para todas las columnas numéricas)
stats = (
    df
    .groupby('Tiene Adiflow')
    .describe(percentiles=[0.25, 0.5, 0.75])
)

# Aplanar columnas tipo (columna, métrica) -> columna_métrica
stats.columns = [f"{col}_{stat}" for col, stat in stats.columns]
stats

,order_count,order_mean,order_std,order_min,order_25%,order_50%,order_75%,order_max,panificadas_count,panificadas_mean,...,dureza_75%,dureza_max,finos_count,finos_mean,finos_std,finos_min,finos_25%,finos_50%,finos_75%,finos_max
Tiene Adiflow,,,,,,,,,,,,,,,,,,,,,
Con Adiflow,287.0,1.001550e+07,292.788035,10014857.0,10015287.5,10015494.0,10015753.5,10016012.0,287.0,78.213240,...,3.2000,3.5,147.0,6.666618,2.193789,1.0,5.333333,6.8,8.05,14.9
Sin Adiflow,191.0,1.001543e+07,380.500936,10014855.0,10015105.5,10015401.0,10015790.5,10016034.0,191.0,49.112042,...,3.1875,4.3,66.0,5.997803,2.952468,0.3,4.300000,6.4,8.00,10.3


In [74]:
df_cut_group_month = compute_sackoff(df_cut, cols=["month"])        # lista ok
df_cut_group_month

,month,Tiene Adiflow,diferencia,reales,production,anulation,sackoff_mean,ops,temp1_acond_c,pdi,pdi_agro,finos,finos_agro,dureza,dureza_agro,sackoff
0,2025-09,Con Adiflow,-5.343609,4277.010709,4240.599609,0.000,0.003046,52,74.303205,91.937821,100.166426,8.008333,7.779851,3.059295,3.071147,-0.124938
1,2025-09,Sin Adiflow,-35.934607,4250.752607,4260.752607,63.210,-0.003901,78,69.204167,92.266667,93.644419,7.772222,6.266566,3.028611,4.450354,-0.845370
2,2025-10,Con Adiflow,-54.706258,8373.738508,8319.481258,36.952,-0.232223,111,74.622082,93.515467,93.611565,6.492866,6.355360,3.149664,3.234853,-0.653307
3,2025-10,Sin Adiflow,-13.555748,920.843748,920.843748,0.000,-0.512103,33,72.550000,94.291667,93.749000,5.708333,6.251000,3.223333,3.071333,-1.472101
4,2025-11,Con Adiflow,-47.489442,7158.510398,6998.702442,0.000,-0.476465,81,74.885522,93.734353,93.676183,6.239403,6.335182,3.097289,3.133742,-0.663398
5,2025-11,Sin Adiflow,-12.434534,1635.846534,1635.846534,0.000,-0.398255,38,74.932000,95.190133,95.031358,4.795867,5.000000,3.106000,3.109259,-0.760128
6,2025-12,Con Adiflow,-26.042535,2487.884835,2470.526535,6.877,-0.274547,36,NaN,NaN,93.936556,NaN,6.068722,NaN,3.167000,-1.046774
7,2025-12,Sin Adiflow,-34.523366,2335.733366,2335.733366,0.000,-0.982825,36,70.000000,98.600000,94.846800,1.400000,5.113200,3.000000,3.151067,-1.478053


In [75]:
cols_visual = [
    'month',
    'Tiene Adiflow',
    'ops',
    'production',
    'reales',
    'anulation',
    'sackoff',
    'diferencia',
    'temp1_acond_c',
    'pdi',
    'finos',
    'dureza']
summary_month = df_cut_group_month.copy()
for col in cols_visual:
    if col not in ['Tiene Adiflow', 'month']:
        summary_month[col] = pd.to_numeric(summary_month[col], errors='coerce')
summary_month = summary_month[cols_visual].round(2)
summary_month = summary_month.rename(columns={
    'ops': 'OPs',
    'production': 'Planificadas (Ton)',
     'reales': 'Producidas (Ton)',
    'anulation': 'Anuladas (Ton)',
    'sackoff': 'Sackoff (%)',
    'diferencia': "Diferencia (Ton)",
    'temp1_acond_c': 'Temp (°C)',
    'pdi': 'Pdi (%)',
    'finos': 'Finos (%)',
    'dureza': 'Dureza (Kg/cm2)',
})
s3.save_dataframe(summary_month, 'summary_month.csv')
summary_month

,month,Tiene Adiflow,OPs,Planificadas (Ton),Producidas (Ton),Anuladas (Ton),Sackoff (%),Diferencia (Ton),Temp (°C),Pdi (%),Finos (%),Dureza (Kg/cm2)
0,2025-09,Con Adiflow,52,4240.60,4277.01,0.00,-0.12,-5.34,74.30,91.94,8.01,3.06
1,2025-09,Sin Adiflow,78,4260.75,4250.75,63.21,-0.85,-35.93,69.20,92.27,7.77,3.03
2,2025-10,Con Adiflow,111,8319.48,8373.74,36.95,-0.65,-54.71,74.62,93.52,6.49,3.15
3,2025-10,Sin Adiflow,33,920.84,920.84,0.00,-1.47,-13.56,72.55,94.29,5.71,3.22
4,2025-11,Con Adiflow,81,6998.70,7158.51,0.00,-0.66,-47.49,74.89,93.73,6.24,3.10
5,2025-11,Sin Adiflow,38,1635.85,1635.85,0.00,-0.76,-12.43,74.93,95.19,4.80,3.11
6,2025-12,Con Adiflow,36,2470.53,2487.88,6.88,-1.05,-26.04,NaN,NaN,NaN,NaN
7,2025-12,Sin Adiflow,36,2335.73,2335.73,0.00,-1.48,-34.52,70.00,98.60,1.40,3.00


In [76]:
df_cut_group_month

,month,Tiene Adiflow,diferencia,reales,production,anulation,sackoff_mean,ops,temp1_acond_c,pdi,pdi_agro,finos,finos_agro,dureza,dureza_agro,sackoff
0,2025-09,Con Adiflow,-5.343609,4277.010709,4240.599609,0.000,0.003046,52,74.303205,91.937821,100.166426,8.008333,7.779851,3.059295,3.071147,-0.124938
1,2025-09,Sin Adiflow,-35.934607,4250.752607,4260.752607,63.210,-0.003901,78,69.204167,92.266667,93.644419,7.772222,6.266566,3.028611,4.450354,-0.845370
2,2025-10,Con Adiflow,-54.706258,8373.738508,8319.481258,36.952,-0.232223,111,74.622082,93.515467,93.611565,6.492866,6.355360,3.149664,3.234853,-0.653307
3,2025-10,Sin Adiflow,-13.555748,920.843748,920.843748,0.000,-0.512103,33,72.550000,94.291667,93.749000,5.708333,6.251000,3.223333,3.071333,-1.472101
4,2025-11,Con Adiflow,-47.489442,7158.510398,6998.702442,0.000,-0.476465,81,74.885522,93.734353,93.676183,6.239403,6.335182,3.097289,3.133742,-0.663398
5,2025-11,Sin Adiflow,-12.434534,1635.846534,1635.846534,0.000,-0.398255,38,74.932000,95.190133,95.031358,4.795867,5.000000,3.106000,3.109259,-0.760128
6,2025-12,Con Adiflow,-26.042535,2487.884835,2470.526535,6.877,-0.274547,36,NaN,NaN,93.936556,NaN,6.068722,NaN,3.167000,-1.046774
7,2025-12,Sin Adiflow,-34.523366,2335.733366,2335.733366,0.000,-0.982825,36,70.000000,98.600000,94.846800,1.400000,5.113200,3.000000,3.151067,-1.478053


In [77]:
summary_month_table = build_monthly_summary_table(
    summary_month,
    month_order=sorted(summary_month['month'].unique()),
)
s3.save_dataframe(summary_month_table, 'summary_month_table.csv')
summary_month_table

,Mes,Sackoff Sin Adiflow,Sackoff Con Adiflow,Diferencia Sackoff,Toneladas Producidas Con Adiflow,Toneladas Recuperadas
0,2025-09,-0.85,-0.12,0.73,4277.01,31.22
1,2025-10,-1.47,-0.65,0.82,8373.74,68.66
2,2025-11,-0.76,-0.66,0.10,7158.51,7.16
3,2025-12,-1.48,-1.05,0.43,2487.88,10.70


In [78]:
m = df_cut_group_month["month"].dt.to_timestamp()
df_cut_group_month["month_lbl"] = m.dt.strftime("%b-%Y")
cats = pd.date_range(m.min(), m.max(), freq="MS").strftime("%b-%Y")
df_cut_group_month["month_lbl"] = pd.Categorical(df_cut_group_month["month_lbl"], categories=cats, ordered=True)
df_cut_group_month

,month,Tiene Adiflow,diferencia,reales,production,anulation,sackoff_mean,ops,temp1_acond_c,pdi,pdi_agro,finos,finos_agro,dureza,dureza_agro,sackoff,month_lbl
0,2025-09,Con Adiflow,-5.343609,4277.010709,4240.599609,0.000,0.003046,52,74.303205,91.937821,100.166426,8.008333,7.779851,3.059295,3.071147,-0.124938,Sep-2025
1,2025-09,Sin Adiflow,-35.934607,4250.752607,4260.752607,63.210,-0.003901,78,69.204167,92.266667,93.644419,7.772222,6.266566,3.028611,4.450354,-0.845370,Sep-2025
2,2025-10,Con Adiflow,-54.706258,8373.738508,8319.481258,36.952,-0.232223,111,74.622082,93.515467,93.611565,6.492866,6.355360,3.149664,3.234853,-0.653307,Oct-2025
3,2025-10,Sin Adiflow,-13.555748,920.843748,920.843748,0.000,-0.512103,33,72.550000,94.291667,93.749000,5.708333,6.251000,3.223333,3.071333,-1.472101,Oct-2025
4,2025-11,Con Adiflow,-47.489442,7158.510398,6998.702442,0.000,-0.476465,81,74.885522,93.734353,93.676183,6.239403,6.335182,3.097289,3.133742,-0.663398,Nov-2025
5,2025-11,Sin Adiflow,-12.434534,1635.846534,1635.846534,0.000,-0.398255,38,74.932000,95.190133,95.031358,4.795867,5.000000,3.106000,3.109259,-0.760128,Nov-2025
6,2025-12,Con Adiflow,-26.042535,2487.884835,2470.526535,6.877,-0.274547,36,NaN,NaN,93.936556,NaN,6.068722,NaN,3.167000,-1.046774,Dec-2025
7,2025-12,Sin Adiflow,-34.523366,2335.733366,2335.733366,0.000,-0.982825,36,70.000000,98.600000,94.846800,1.400000,5.113200,3.000000,3.151067,-1.478053,Dec-2025


In [79]:
f = plot_bar(
    df_cut_group_month.round(2),
    x_col="month_lbl",
    y_col="sackoff",
    group_col="Tiene Adiflow",
    order_x=sorted(df_cut_group_month["month_lbl"].unique(), reverse=True),
    title="Sackoff Total por mes",
    cat_base="Sin Adiflow",
    show_delta=True,
    x_title="Mes",
    y_title="Sackoff",
    text_format=".2f",
    delta_unit="%",
    hover_data_cols=['reales', 'production'],
    height=500,
    width=1000,
)
f.show()
s3.save_plotly_html(f, "barras_sackoff_mes.html")

In [80]:
df_group_dieta_ = compute_sackoff(df_both_cut, cols=["Dieta"])        # lista ok
df_group_dieta_

,Dieta,Tiene Adiflow,diferencia,reales,production,anulation,sackoff_mean,ops,temp1_acond_c,pdi,pdi_agro,finos,finos_agro,dureza,dureza_agro,sackoff
0,ENGORDE ESP P,Con Adiflow,-0.229702,1083.730002,1073.833702,0.000,-0.418632,11,75.625000,93.087500,93.425000,6.970833,6.571296,3.084167,3.105000,-0.021195
1,ENGORDE ESP P,Sin Adiflow,-6.198465,602.710465,602.710465,27.520,-0.747725,7,73.475000,92.125000,93.302778,7.875000,6.680556,3.087500,3.202778,-1.028432
2,FINALIZACION,Con Adiflow,-19.406636,3186.224636,3160.980636,15.250,-0.298929,34,75.275500,93.423333,93.711979,6.545417,6.313906,3.156667,3.176354,-0.609079
3,FINALIZACION,Sin Adiflow,-0.870430,90.100430,90.100430,0.000,-0.966066,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.966066
4,FLUSHING,Con Adiflow,-1.582306,326.245606,324.512306,0.000,0.091724,11,70.933333,95.680000,96.225000,4.320000,3.775000,3.146000,3.169583,-0.485005
5,FLUSHING,Sin Adiflow,-3.398455,242.353455,242.353455,0.000,-2.473284,7,74.000000,93.566667,95.780000,6.433333,4.220000,3.133333,3.160000,-1.402272
6,GESTACION,Con Adiflow,-16.966310,3535.455960,3511.370310,0.000,-0.439392,31,77.679273,91.087218,91.056807,8.882957,8.858824,3.122607,3.137137,-0.479890
7,GESTACION,Sin Adiflow,-6.844236,535.216236,535.216236,0.000,-0.908962,5,80.000000,90.000000,92.010000,10.000000,7.990000,3.000000,3.200000,-1.278780
8,INICIACION P INMUNIDAD,Con Adiflow,-36.345215,3260.953515,3245.525215,0.000,-1.096604,25,71.913646,95.219271,95.506267,4.783854,4.457400,3.127083,3.490340,-1.114558
9,INICIACION P INMUNIDAD,Sin Adiflow,-18.395146,2416.045146,2416.045146,0.000,0.637085,19,73.525926,94.081481,95.287018,6.140741,4.686667,3.038889,3.060351,-0.761374


In [81]:
# Partimos de df_both (ya solo dietas con ambos estados)
pivot = (
    df_group_dieta_
    .pivot_table(
        index="Dieta",
        columns="Tiene Adiflow",
        values="sackoff",
        aggfunc="mean"   # o 'sum' según cómo definas
    )
)

# Dietas donde CON Adiflow tiene mejor sackoff (= valor más alto, menos pérdida)
df_mejor_con = (
    pivot[pivot["Con Adiflow"] > pivot["Sin Adiflow"]]
    .assign(delta=lambda x: x["Con Adiflow"] - x["Sin Adiflow"])
    .sort_values("delta", ascending=False)
    .reset_index()
)

df_mejor_con = df_mejor_con.round(2)
s3.save_dataframe(df_mejor_con, "mejores_resultados.csv")
df_mejor_con

Tiene Adiflow,Dieta,Con Adiflow,Sin Adiflow,delta
0,LACTANCIA PRIMERIZAS ESP,-0.67,-4.87,4.20
1,MACHOS ESP 113,1.02,-2.35,3.38
2,PRELACTANCIA H,0.50,-2.27,2.77
3,REEMPLAZOS S1B,0.45,-1.89,2.34
4,PREINICIACION F2 P INMUNIDAD,1.30,-0.67,1.96
5,NOVILLONAS SUPLEMENTO,-0.18,-1.93,1.75
6,LEVANTE CMC,-0.24,-1.75,1.51
7,ENGORDE ESP P,-0.02,-1.03,1.01
8,FLUSHING,-0.49,-1.40,0.92
9,TERNERAS H,-0.40,-1.31,0.91


In [82]:
cols_visual = [
    'Dieta',
    'Tiene Adiflow',
    'ops',
    'production',
    'reales',
    'anulation',
    'sackoff',
    'diferencia',
    'temp1_acond_c',
    'pdi',
    'finos',
    'dureza']
summary_dieta = df_group_dieta_.copy()
for col in cols_visual:
    if col not in ['Tiene Adiflow', 'Dieta']:
        summary_dieta[col] = pd.to_numeric(summary_dieta[col], errors='coerce')
summary_dieta = summary_dieta[cols_visual].round(2)
summary_dieta = summary_dieta.rename(columns={
    'ops': 'OPs',
    'production': 'Planificadas (Ton)',
     'reales': 'Producidas (Ton)',
    'anulation': 'Anuladas (Ton)',
    'sackoff': 'Sackoff (%)',
    'diferencia': "Diferencia (Ton)",
    'temp1_acond_c': 'Temp (°C)',
    'pdi': 'Pdi (%)',
    'finos': 'Finos (%)',
    'dureza': 'Dureza (Kg/cm2)',
})
summary_dieta = summary_dieta[summary_dieta["Dieta"].isin(df_mejor_con["Dieta"])]
s3.save_dataframe(summary_dieta, "summary_dieta.csv")
summary_dieta

,Dieta,Tiene Adiflow,OPs,Planificadas (Ton),Producidas (Ton),Anuladas (Ton),Sackoff (%),Diferencia (Ton),Temp (°C),Pdi (%),Finos (%),Dureza (Kg/cm2)
0,ENGORDE ESP P,Con Adiflow,11,1073.83,1083.73,0.00,-0.02,-0.23,75.62,93.09,6.97,3.08
1,ENGORDE ESP P,Sin Adiflow,7,602.71,602.71,27.52,-1.03,-6.20,73.47,92.12,7.88,3.09
2,FINALIZACION,Con Adiflow,34,3160.98,3186.22,15.25,-0.61,-19.41,75.28,93.42,6.55,3.16
3,FINALIZACION,Sin Adiflow,1,90.10,90.10,0.00,-0.97,-0.87,NaN,NaN,NaN,NaN
4,FLUSHING,Con Adiflow,11,324.51,326.25,0.00,-0.49,-1.58,70.93,95.68,4.32,3.15
5,FLUSHING,Sin Adiflow,7,242.35,242.35,0.00,-1.40,-3.40,74.00,93.57,6.43,3.13
6,GESTACION,Con Adiflow,31,3511.37,3535.46,0.00,-0.48,-16.97,77.68,91.09,8.88,3.12
7,GESTACION,Sin Adiflow,5,535.22,535.22,0.00,-1.28,-6.84,80.00,90.00,10.00,3.00
12,LACTANCIA PRIMERIZAS ESP,Con Adiflow,17,1104.30,1110.63,0.00,-0.67,-7.49,71.64,94.49,5.51,3.08
13,LACTANCIA PRIMERIZAS ESP,Sin Adiflow,1,18.06,18.06,0.00,-4.87,-0.88,NaN,NaN,NaN,NaN


In [83]:
summary_month_table = build_monthly_summary_table(
    summary_dieta,
     month_col= "Dieta",
    month_order=df_mejor_con["Dieta"].unique().tolist(),
)
summary_month_table.rename(columns={"Mes": "Dieta"}, inplace=True)
summary_month_table = summary_month_table.sort_values("Toneladas Recuperadas", ascending=False)
s3.save_dataframe(summary_month_table, f"summary_dieta_table.csv")
summary_month_table

,Dieta,Sackoff Sin Adiflow,Sackoff Con Adiflow,Diferencia Sackoff,Toneladas Producidas Con Adiflow,Toneladas Recuperadas
0,LACTANCIA PRIMERIZAS ESP,-4.87,-0.67,4.20,1110.63,46.65
10,GESTACION,-1.28,-0.48,0.80,3535.46,28.28
13,FINALIZACION,-0.97,-0.61,0.36,3186.22,11.47
7,ENGORDE ESP P,-1.03,-0.02,1.01,1083.73,10.95
4,PREINICIACION F2 P INMUNIDAD,-0.67,1.30,1.97,462.38,9.11
15,LEVANTE,-0.55,-0.26,0.29,3001.46,8.70
14,LEVANTE ESP P,-0.37,-0.05,0.32,2182.45,6.98
6,LEVANTE CMC,-1.75,-0.24,1.51,233.33,3.52
8,FLUSHING,-1.40,-0.49,0.91,326.25,2.97
12,LACTANCIA SILO H,-0.84,-0.24,0.60,404.66,2.43


In [84]:
f = plot_bar(
    df_group_dieta_.round(2),
    x_col="Dieta",
    y_col="sackoff",
    group_col="Tiene Adiflow",
    order_x=["Septiembre", "Octubre", "Noviembre", "Diciembre"],
    title="Sackoff Total por Dieta",
    cat_base="Sin Adiflow",
    show_delta=True,
    x_title="Mes",
    y_title="Sackoff",
    text_format=".2f",
    delta_unit="%",
    hover_data_cols=['reales', 'production'],
    height=500,
    width=1000,
)
f.show()
s3.save_plotly_html(f, "barras_sackoff_dieta_both.html")

In [85]:
df_group_dieta_mes = compute_sackoff(df_both_cut, cols=["Dieta", "month"])        # lista ok


In [86]:
months = pd.PeriodIndex(["2025-11", "2025-12"], freq="M")
df_cut_nov = df_cut[df_cut["month"].isin(months)]
compute_sackoff(df_cut_nov, cols=[])

,Tiene Adiflow,diferencia,reales,production,anulation,sackoff_mean,ops,temp1_acond_c,pdi,pdi_agro,finos,finos_agro,dureza,dureza_agro,sackoff
0,Con Adiflow,-73.531977,9646.395233,9469.228977,6.877,-0.429012,117,74.885522,93.734353,93.747845,6.239403,6.261845,3.097289,3.142896,-0.762274
1,Sin Adiflow,-46.957899,3971.579899,3971.579899,0.000,-0.822791,73,74.742308,95.321282,94.942628,4.665256,5.054423,3.101923,3.129359,-1.182348


In [87]:
cols = ['order', 'Dieta', 'op', 'date', 'Tiene Adiflow', 'panificadas', 'entregadas', 'code_101', 'Producción (Ton)',
       'Anulación (Ton)',
       'diff_prod',  'sackoff_prod', 'peso_agua'


]
df_cut_nov = df_cut_nov.sort_values('sackoff_prod', ascending=True)
df_cut_nov_dep = df_cut_nov[cols]
s3.save_dataframe(df_cut_nov_dep,  "datos_noviembre_sackoff_fazenda.csv")
df_cut_nov_dep

,order,Dieta,op,date,Tiene Adiflow,panificadas,entregadas,code_101,Producción (Ton),Anulación (Ton),diff_prod,sackoff_prod,peso_agua
453,10015982.0,REEMPLAZOS S1B,22075,2025-12-09,Sin Adiflow,10.0,8.590,8.590,10.028288,0.0,-0.85230,-0.143423,0.00000
419,10015899.0,INICIACION P INMUNIDAD,22024,2025-12-02,Con Adiflow,64.0,57.368,57.368,64.286253,0.0,-6.91828,-0.107616,0.30005
432,10015942.0,FINALIZACION,22049,2025-12-04,Con Adiflow,8.0,7.220,7.220,8.032005,0.0,-0.81201,-0.101096,0.07400
289,10015608.0,MACHOS ESP 113,21874,2025-11-01,Sin Adiflow,2.0,1.820,1.820,2.002710,0.0,-0.18271,-0.091231,0.00000
433,10015943.0,ENGORDE ESP P,22052,2025-12-04,Sin Adiflow,48.4,44.170,44.170,48.448962,0.0,-4.27896,-0.088319,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
386,10015830.0,LACTANCIA SILO H,21989,2025-11-24,Con Adiflow,36.0,37.300,37.300,36.143500,0.0,1.15650,0.031997,0.33330
456,10015985.0,B. NOVILLOS CEBA,22064,2025-12-10,Sin Adiflow,20.0,20.780,20.780,20.006500,0.0,2.57350,0.038662,0.00000
405,10015875.0,INICIACION P INMUNIDAD,22014,2025-11-29,Con Adiflow,140.0,79.729,79.729,76.361861,0.0,40.87365,0.044095,0.35375
429,10015939.0,ENGORDE ESP P,22045,2025-12-04,Con Adiflow,64.0,72.778,72.778,68.958491,0.0,3.81951,0.055389,0.63170


In [88]:

seg_ = ["Tiene Adiflow"]
q = df_cut_nov.groupby(seg_)["sackoff_op"].quantile(
    [0,0.01, 0.02, 0.10, 0.15, 0.20, 0.30, 0.50, 0.90, 0.98, 0.99,1]
).unstack(level=1)

q.columns = ["min","q01", "q02", "q10", 'q15', 'q20', 'q30', "q50", "q90", "q98", "q99", "max"]
q["mean"] = df_cut_nov.groupby(seg_)["sackoff_op"].mean()
#q["min"] = df_cut_nov.groupby(seg_)["sackoff_op"].min()
#q["max"] = df_cut_nov.groupby(seg_)["sackoff_op"].max()
q

,min,q01,q02,q10,q15,q20,q30,q50,q90,q98,q99,max,mean
Tiene Adiflow,,,,,,,,,,,,,
Con Adiflow,-10.761637,-9.922436,-8.024473,-2.353509,-1.828372,-1.598324,-1.058813,-0.429012,0.902030,3.998151,5.346855,6.243745,-0.618568
Sin Adiflow,-14.342308,-10.532321,-8.989172,-5.015412,-4.287300,-2.844750,-2.022492,-0.822791,0.922859,2.041612,2.914746,3.866243,-1.505837


In [89]:
q_min = df_cut_nov["sackoff_op"].quantile(0.10)
q_max = df_cut_nov["sackoff_op"].quantile(0.99)

cond1 = df_cut_nov["sackoff_op"] >= q_min
cond2 = df_cut_nov["sackoff_op"] <= q_max
cond_range = cond1 & cond2

df_cut_nov_new = df_cut_nov[cond_range].copy()
df_bad_nov_new = df_cut_nov[~cond_range].copy()
df_bad_nov_new


,order,Dieta,op,date,panificadas,entregadas,code_101,Anulación (Ton),code_122,code_309,...,peso_agua,Tiene Adiflow,pdi_agro,dureza_agro,finos_agro,product_name,temp1_acond_c,pdi,dureza,finos
453,10015982.0,REEMPLAZOS S1B,22075,2025-12-09,10.0,8.590,8.590,0.000,0.0,0.0,...,0.00000,Sin Adiflow,91.000000,3.000000,9.000000,NaN,NaN,NaN,NaN,NaN
419,10015899.0,INICIACION P INMUNIDAD,22024,2025-12-02,64.0,57.368,57.368,0.000,0.0,0.0,...,0.30005,Con Adiflow,94.400000,3.100000,5.600000,NaN,NaN,NaN,NaN,NaN
432,10015942.0,FINALIZACION,22049,2025-12-04,8.0,7.220,7.220,0.000,0.0,0.0,...,0.07400,Con Adiflow,94.000000,3.300000,6.000000,NaN,NaN,NaN,NaN,NaN
289,10015608.0,MACHOS ESP 113,21874,2025-11-01,2.0,1.820,1.820,0.000,0.0,0.0,...,0.00000,Sin Adiflow,92.900000,3.100000,7.100000,MACHOS P,90.0,92.90,3.1,7.10
433,10015943.0,ENGORDE ESP P,22052,2025-12-04,48.4,44.170,44.170,0.000,0.0,0.0,...,0.00000,Sin Adiflow,94.200000,3.200000,5.800000,NaN,NaN,NaN,NaN,NaN
423,10015907.0,REEMPLAZOS S1B,22040,2025-12-02,12.0,11.053,11.053,0.000,0.0,0.0,...,0.00000,Sin Adiflow,95.400000,3.400000,4.600000,NaN,NaN,NaN,NaN,NaN
450,10015977.0,FLUSHING,22069,2025-12-09,26.0,26.260,26.260,0.000,0.0,0.0,...,0.12950,Con Adiflow,96.600000,3.100000,3.400000,NaN,NaN,NaN,NaN,NaN
473,10016023.0,NOVILLONAS SUPLEMENTO,22097,2025-12-15,6.0,5.660,5.660,0.000,0.0,0.0,...,0.00000,Sin Adiflow,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
364,10015775.0,LEVANTE CMC,21969,2025-11-18,30.0,28.388,28.388,0.000,0.0,0.0,...,0.00000,Sin Adiflow,94.800000,3.200000,5.200000,LEVANTE CMC,70.0,94.80,3.2,5.20
297,10015620.0,B. NOVILLAS LEVANTE,21883,2025-11-03,4.0,3.780,3.780,0.000,0.0,0.0,...,0.00000,Sin Adiflow,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
